In [28]:
import gym
import numpy as np

In [12]:
env = gym.make('SpaceInvaders-v0')

In [4]:
episodes = 10

for episode in range(1, episodes):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        state, reward, done, info = env.step(env.action_space.sample())
        score += reward
    print('Episode: {}\nScore: {}'.format(episode, score))
    
env.close()

Episode: 1
Score: 160.0
Episode: 2
Score: 135.0
Episode: 3
Score: 155.0
Episode: 4
Score: 140.0
Episode: 5
Score: 135.0
Episode: 6
Score: 30.0
Episode: 7
Score: 375.0
Episode: 8
Score: 260.0
Episode: 9
Score: 105.0


In [13]:
# import Neural Network packages
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [14]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

In [15]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32, 
                     (8,8), 
                     strides=(4,4), 
                     activation='relu', 
                     input_shape =(3, height, width, channels)))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [16]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [21]:
del(model)

In [22]:
model = build_model(height, width, channels, actions)

In [18]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [19]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=2000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg',
                  nb_actions=actions, nb_steps_warmup=1000)
    return dqn


In [23]:
dqn = build_agent(model, actions)

In [24]:
dqn.compile(Adam(lr=0.0001))

In [23]:
dqn.fit(env, nb_steps=4000, visualize=False, verbose=1)

Training for 4000 steps ...
Interval 1 (0 steps performed)


C:\Users\Alex\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 1801/10000 [====>.........................] - ETA: 45:51 - reward: 0.1444done, took 606.522 seconds


In [30]:
scores = dqn.test(env, nb_episodes=1, Visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 1 episodes ...
Episode 1: reward: 60.000, steps: 917
60.0


AttributeError: 'AtariEnv' object has no attribute 'play'

In [34]:
from tensorflow.keras.models import load_model
new_model =load_model('./')

In [37]:
dqn = build_agent(new_model, actions)

In [38]:
dqn.compile(Adam(lr=0.0001))

In [36]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
flatten (Flatten)            (None, 82944)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               42467840  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1542      
Total params: 42,639,718
Trainable params: 42,639,718
Non-trainable params: 0
____________________________________________

In [40]:
scores = dqn.test(env, nb_episodes=1, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 1 episodes ...
Episode 1: reward: 0.000, steps: 514
0.0


In [ ]:
dqn.save_weights('models/dqn.h5f')